## Week 2 Day 3

Now we get to more detail:

1. Different models

2. Structured Outputs

3. Guardrails

In [24]:
from dotenv import load_dotenv
from openai import AsyncOpenAI
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel, input_guardrail, GuardrailFunctionOutput
from typing import Dict, List
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
from pydantic import BaseModel

In [2]:
load_dotenv(override=True)

True

In [3]:
openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

if deepseek_api_key:
    print(f"DeepSeek API Key exists and begins {deepseek_api_key[:3]}")
else:
    print("DeepSeek API Key not set (and this is optional)")

if groq_api_key:
    print(f"Groq API Key exists and begins {groq_api_key[:4]}")
else:
    print("Groq API Key not set (and this is optional)")

OpenAI API Key exists and begins sk-proj-
Google API Key exists and begins AI
DeepSeek API Key exists and begins sk-
Groq API Key exists and begins gsk_


In [4]:
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

### It's easy to use any models with OpenAI compatible endpoints

In [5]:
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
DEEPSEEK_BASE_URL = "https://api.deepseek.com/v1"
GROQ_BASE_URL = "https://api.groq.com/openai/v1"

In [6]:

deepseek_client = AsyncOpenAI(base_url=DEEPSEEK_BASE_URL, api_key=deepseek_api_key)
gemini_client = AsyncOpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)
groq_client = AsyncOpenAI(base_url=GROQ_BASE_URL, api_key=groq_api_key)

deepseek_model = OpenAIChatCompletionsModel(model="deepseek-chat", openai_client=deepseek_client)
gemini_model = OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=gemini_client)
llama3_3_model = OpenAIChatCompletionsModel(model="llama-3.3-70b-versatile", openai_client=groq_client)

In [7]:
sales_agent1 = Agent(name="DeepSeek Sales Agent", instructions=instructions1, model=deepseek_model)
sales_agent2 =  Agent(name="Gemini Sales Agent", instructions=instructions2, model=gemini_model)
sales_agent3  = Agent(name="Llama3.3 Sales Agent",instructions=instructions3,model=llama3_3_model)

In [8]:
description = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

In [9]:
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("williamlapa40@gmail.com")  # Change to your verified sender
    to_email = To("williamlapa@hotmail.com")  # Change to your recipient
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [10]:
subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")

In [11]:
email_tools = [subject_tool, html_tool, send_html_email]

In [12]:
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."


emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=email_tools,
    model="gpt-4o-mini",
    handoff_description="Convert an email to HTML and send it")

In [13]:
tools = [tool1, tool2, tool3]
handoffs = [emailer_agent]

In [15]:
sales_manager_instructions = "You are a sales manager working for ComplAI. You use the tools given to you to generate cold sales emails. \
You never generate sales emails yourself; you always use the tools. \
You try all 3 sales agent tools at least once before choosing the best one. \
You can use the tools multiple times if you're not satisfied with the results from the first try. \
You select the single best email using your own judgement of which email will be most effective. \
After picking the email, you handoff to the Email Manager agent to format and send the email.\
Gere os textos em português "


sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model="gpt-4o-mini")

message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Automated SDR"):
    result = await Runner.run(sales_manager, message)

In [19]:
print(result.final_output)

The cold sales email has been successfully sent to the CEO. If you need further assistance or have more emails to send, just let me know!


## Check out the trace:

https://platform.openai.com/traces

In [20]:
class NameCheckOutput(BaseModel):
    is_name_in_message: bool
    name: str

guardrail_agent = Agent( 
    name="Name check",
    instructions="Check if the user is including someone's personal name in what they want you to do.",
    output_type=NameCheckOutput,
    model="gpt-4o-mini"
)

In [21]:
@input_guardrail
async def guardrail_against_name(ctx, agent, message):
    result = await Runner.run(guardrail_agent, message, context=ctx.context)
    is_name_in_message = result.final_output.is_name_in_message
    return GuardrailFunctionOutput(output_info={"found_name": result.final_output},tripwire_triggered=is_name_in_message)

In [22]:
careful_sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=[emailer_agent],
    model="gpt-4o-mini",
    input_guardrails=[guardrail_against_name]
    )

message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Protected Automated SDR"):
    result = await Runner.run(careful_sales_manager, message)

InputGuardrailTripwireTriggered: Guardrail InputGuardrail triggered tripwire

## Check out the trace:

https://platform.openai.com/traces

In [23]:

message = "Send out a cold sales email addressed to Dear CEO from Head of Business Development"

with trace("Protected Automated SDR with no names"):
    result = await Runner.run(careful_sales_manager, message)

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercise</h2>
            <span style="color:#ff7800;">• Try different models<br/>• Add more input and output guardrails<br/>• Use structured outputs for the email generation
            </span>
        </td>
    </tr>
</table>

## Outro exemplo de uso do guardrail

In [ ]:
# Guardrail para verificação de senha comum

# 1. O que o Agente Guardrail Vai Entregar
class PasswordCheckOutput(BaseModel):
    has_common_password: bool
    found_passwords: List[str]

# 2. O Agente de Verificação de Senhas
password_guardrail_agent = Agent(
    name="Password Checker",
    instructions="Check if the user's message contains any common or obvious passwords like 'password', '123456', 'admin', etc.",
    output_type=PasswordCheckOutput,
    model="gpt-4o-mini"
)

# 3. A Função da Barreira de Segurança
@input_guardrail
async def guardrail_against_passwords(ctx, agent, message):
    result = await Runner.run(password_guardrail_agent, message, context=ctx.context)
    has_password_in_message = result.final_output.has_common_password
    return GuardrailFunctionOutput(output_info={"found_common_password": result.final_output}, tripwire_triggered=has_password_in_message)

# 4. O Agente Principal (que vai usar o guardrail)
# Vamos reutilizar o 'careful_sales_manager' e adicionar o novo guardrail
# Observação: Para este exemplo, 'sales_manager_instructions', 'tools' e 'emailer_agent'
# seriam definidos em outro lugar no seu código, como no exemplo anterior.
# Apenas a lista de 'input_guardrails' é modificada aqui.

class SomeAgentOutput(BaseModel): # Apenas um tipo de saída de exemplo para o agente principal
    status: str

careful_sales_manager = Agent(
    name="Sales Manager",
    instructions="Manage sales tasks and interact with customers.",
    output_type=SomeAgentOutput, # Usando um tipo de saída de exemplo
    tools=tools, 
    handoffs=[emailer_agent], 
    model="gpt-4o-mini",
    input_guardrails=[guardrail_against_passwords] # Adicionando o novo guardrail
)

# 5. Mensagens de Exemplo
message_with_password = "Please reset my account. My new password is password123."
message_without_password = "I need to send an email to John Doe."

# Executando com o guardrail
async def run_example():
    print("--- Testando com senha ---")
    with trace("Protected Automated SDR"):
        result_with_password = await Runner.run(careful_sales_manager, message_with_password)
        print(f"Resultado com senha: {result_with_password.final_output}")
        print(f"Guardrail ativado: {result_with_password.tripwire_triggered}")
        print(f"Detalhes do guardrail: {result_with_password.guardrails_output}")

    print("\n--- Testando sem senha ---")
    with trace("Protected Automated SDR"):
        result_without_password = await Runner.run(careful_sales_manager, message_without_password)
        print(f"Resultado sem senha: {result_without_password.final_output}")
        print(f"Guardrail ativado: {result_without_password.tripwire_triggered}")
        print(f"Detalhes do guardrail: {result_without_password.guardrails_output}")


await run_example()

In [ ]:
# Executando exemplo
message = "Please reset my account. My new password is 123456."

with trace("Exeemplo Guardrail com senha"):
    result = await Runner.run(careful_sales_manager, message)
    print(f"Resultado com senha: {result.final_output}")
    # print(f"Guardrail ativado: {result.tripwire_triggered}")
    # print(f"Detalhes do guardrail: {result.guardrails_output}")

## Usando outra estrutura de resposta

In [33]:
# Definindo a Estrutura do E-mail (EmailOutput)

from pydantic import BaseModel, Field
from typing import List, Optional

class EmailOutput(BaseModel):
    """Estrutura para um e-mail de vendas."""
    recipient_name: str = Field(description="O nome do destinatário do e-mail.")
    recipient_title: Optional[str] = Field(description="O cargo do destinatário, se conhecido. Opcional.")
    subject: str = Field(description="O assunto conciso e cativante do e-mail.")
    greeting: str = Field(description="A saudação personalizada do e-mail (ex: 'Prezado [Nome]', 'Olá [Nome]').")
    body_paragraphs: List[str] = Field(description="Uma lista de parágrafos que compõem o corpo do e-mail. Cada item da lista é um parágrafo.")
    call_to_action: str = Field(description="Uma chamada clara à ação para o destinatário (ex: 'Agende uma reunião', 'Visite nosso site').")
    sender_name: str = Field(description="O nome do remetente do e-mail.")
    sender_title: str = Field(description="O cargo do remetente do e-mail.")
    signature: str = Field(description="A assinatura do e-mail (ex: 'Atenciosamente,', 'Cordialmente,').")
    company_name: str = Field(description="O nome da empresa que envia o e-mail.")

In [34]:
# Agente de Geração de E-mails
# Agora, vamos criar um agente que usará essa estrutura para gerar o e-mail.

# As definições de NameCheckOutput e guardrail_against_name do exemplo anterior
# (se você quiser usar o guardrail de nome neste agente também)
class NameCheckOutput(BaseModel):
    is_name_in_message: bool
    name: str

guardrail_agent = Agent(
    name="Name check",
    instructions="Check if the user is including someone's personal name in what they want you to do.",
    output_type=NameCheckOutput,
    model="gpt-4o-mini"
)

@input_guardrail
async def guardrail_against_name(ctx, agent, message):
    result = await Runner.run(guardrail_agent, message, context=ctx.context)
    is_name_in_message = result.final_output.is_name_in_message
    # Se o nome for detectado, o tripwire é ativado, e a execução do agente principal pode ser interrompida ou alterada.
    return GuardrailFunctionOutput(output_info={"found_name": result.final_output},tripwire_triggered=is_name_in_message)

# Agora, o agente de geração de e-mails
emailer_agent = Agent(
    name="Email Generator",
    instructions="""Você é um redator de e-mails de vendas profissional.
    Sua tarefa é gerar um e-mail de vendas convincente para um novo lead,
    com base nas informações fornecidas.
    Seja conciso, profissional e persuasivo.
    Sua saída DEVE seguir o formato EmailOutput estritamente.
    """,
    output_type=EmailOutput, # AQUI está a chave: definimos a saída esperada
    model="gpt-4o-mini", # Pode ser 'gpt-4o', 'gpt-3.5-turbo', etc.
    # input_guardrails=[guardrail_against_name] # Podemos aplicar o guardrail de nome aqui também, se quisermos.
)

# O agente principal que orquestra
sales_manager_instructions = """
Você é um gerente de vendas automatizado.
Sua principal função é receber solicitações de leads e gerar e-mails de vendas.
Use o 'Email Generator' para criar o e-mail.
"""

careful_sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    handoffs=[emailer_agent], # O Sales Manager pode "passar a bola" para o Emailer Agent
    model="gpt-4o-mini",
    input_guardrails=[guardrail_against_name] # Reutilizando o guardrail de nome
)

In [35]:
# A mensagem que o usuário daria ao Sales Manager
message = "Gerar um e-mail de vendas para João Silva, CEO da TechInovate, sobre nossa nova solução de IA para otimização de processos."

# Executando o processo
async def run_email_generation_example():
    print("--- Gerando E-mail com Saída Estruturada ---")
    with trace("Email Generation SDR"):
        # O Sales Manager receberá a mensagem e fará o handoff para o Emailer Agent
        result = await Runner.run(careful_sales_manager, message)

        # O 'final_output' do Runner.run quando há um handoff para um agente com output_type
        # conterá a saída estruturada desse agente.
        # Guardrails-AI pode encapsular isso um pouco, então precisamos acessar corretamente.

        # Verificando se o tripwire foi ativado (se o guardrail de nome estivesse ativo e detectasse um nome)
        if result.tripwire_triggered:
            print(f"Guardrail Ativado! Motivo: {result.guardrails_output.get('guardrail_against_name').output_info}")
            print("Não foi possível gerar o e-mail devido ao guardrail.")
            return

        # A saída final do 'Runner.run' para o 'careful_sales_manager'
        # deve conter o resultado do 'emailer_agent' se o handoff foi bem-sucedido.
        # A forma exata de acessar depende da versão da biblioteca, mas geralmente é via final_output
        # ou através de logs detalhados.
        # No guardrails-ai, se um handoff ocorre e o agente delegado produz uma saída,
        # essa saída é o 'final_output' do Runner.run.

        email_data: EmailOutput = result.final_output

        print("\n--- E-mail Gerado (Estruturado) ---")
        print(f"Tipo de objeto: {type(email_data)}")
        print(f"Nome do Destinatário: {email_data.recipient_name}")
        print(f"Cargo do Destinatário: {email_data.recipient_title}")
        print(f"Assunto: {email_data.subject}")
        print(f"Saudação: {email_data.greeting}")
        print("Corpo do E-mail:")
        for paragraph in email_data.body_paragraphs:
            print(f"- {paragraph}")
        print(f"Chamada para Ação: {email_data.call_to_action}")
        print(f"Nome do Remetente: {email_data.sender_name}")
        print(f"Cargo do Remetente: {email_data.sender_title}")
        print(f"Assinatura: {email_data.signature}")
        print(f"Nome da Empresa: {email_data.company_name}")

        # Como você usaria isso em um sistema de envio de e-mail:
        print("\n--- Exemplo de Montagem do E-mail Real ---")
        full_email_body = "\n\n".join(email_data.body_paragraphs)
        final_email_text = (
            f"Para: {email_data.recipient_name} ({email_data.recipient_title or 'N/A'})\n"
            f"Assunto: {email_data.subject}\n\n"
            f"{email_data.greeting}\n\n"
            f"{full_email_body}\n\n"
            f"{email_data.call_to_action}\n\n"
            f"{email_data.signature}\n"
            f"{email_data.sender_name}\n"
            f"{email_data.sender_title}\n"
            f"{email_data.company_name}"
        )
        print(final_email_text)

# Para rodar (se for um script Python):
if __name__ == "__main__":
    # Mock para evitar erros se tools e emailer_agent não estiverem definidos globalmente
    class MockTool:
        def __call__(self, *args, **kwargs):
            pass
    tools = [MockTool()]
    class MockEmailerAgent: # Ajustando o mock para o handoff
        def __init__(self, *args, **kwargs):
            self.name = kwargs.get('name', 'MockEmailer')
            self.instructions = kwargs.get('instructions', '')
            self.output_type = kwargs.get('output_type')
            self.model = kwargs.get('model', 'gpt-4o-mini') # Mock LLM model

        async def run(self, message, context):
            # Este é um mock simplificado de como o EmailerAgent real responderia
            # Em um cenário real, você não precisaria mockar o EmailerAgent se ele estivesse definido corretamente
            # Este mock é apenas para que o `Runner.run` do `careful_sales_manager` funcione sem erros de dependência
            return {
                "final_output": EmailOutput(
                    recipient_name="João Silva",
                    recipient_title="CEO",
                    subject="Potencialize a Otimização de Processos com Nossa Solução de IA",
                    greeting="Prezado João,",
                    body_paragraphs=[
                        "Entendemos que a otimização de processos é crucial para a inovação e o crescimento.",
                        "Nossa nova solução de IA foi desenvolvida para identificar gargalos, automatizar tarefas repetitivas e gerar insights acionáveis, resultando em maior eficiência e redução de custos para a TechInovate.",
                        "Gostaria de explorar como podemos adaptar essa tecnologia às necessidades específicas de sua empresa."
                    ],
                    call_to_action="Agende uma breve demonstração para ver nossa solução em ação e discutir o potencial impacto na TechInovate.",
                    sender_name="Seu Nome",
                    sender_title="Gerente de Contas",
                    signature="Atenciosamente,",
                    company_name="Sua Empresa de Inovação"
                )
            }

    # Re-definir o emailer_agent usando a classe MockEmailerAgent para evitar erros de dependência
    # APENAS PARA FINS DE TESTE DESTE SCRIPT ISOLADO
    # Em um projeto real, você teria a definição completa do EmailerAgent
    emailer_agent = Agent(
        name="Email Generator",
        instructions="""Você é um redator de e-mails de vendas profissional.
        Sua tarefa é gerar um e-mail de vendas convincente para um novo lead,
        com base nas informações fornecidas.
        Seja conciso, profissional e persuasivo.
        Sua saída DEVE seguir o formato EmailOutput estritamente.
        """,
        output_type=EmailOutput, # AQUI está a chave: definimos a saída esperada
        model="gpt-4o-mini"
    )

    # Re-definir o careful_sales_manager para ter o handoff correto
    careful_sales_manager = Agent(
        name="Sales Manager",
        instructions=sales_manager_instructions,
        handoffs=[emailer_agent], # O Sales Manager pode "passar a bola" para o Emailer Agent
        model="gpt-4o-mini",
        input_guardrails=[guardrail_against_name] # Reutilizando o guardrail de nome
    )

    await run_email_generation_example()

--- Gerando E-mail com Saída Estruturada ---


C:\Users\02392179425\AppData\Local\Programs\Python\Python312\Lib\re\__init__.py:260: RuntimeWarning: coroutine 'run_example' was never awaited
  return pattern.translate(_special_chars_map)


InputGuardrailTripwireTriggered: Guardrail InputGuardrail triggered tripwire